In [39]:
import pandas as pd

In [40]:
df = pd.DataFrame({'text':['I like kaggle very much',
                           'I do not like kaggle',
                           'I do really love machine learning']})


문장을 벡터로 변환 (특징 부여)

In [41]:
# 단어 수 세기 'bag of words'
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
bag= vectorizer.fit_transform(df['text'])
bag.toarray()

array([[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]], dtype=int64)

In [42]:
vectorizer.vocabulary_ #각 단어의 index번호

{'i': 1,
 'like': 4,
 'kaggle': 2,
 'very': 10,
 'much': 7,
 'do': 0,
 'not': 8,
 'really': 9,
 'love': 5,
 'machine': 6,
 'learning': 3}

bag of words의 장점: 이해하기 쉬움

단점:

- 특별한 의미 x

- 단어끼리의 인접성 확인 불가

- 단어 순서와 관련된 정보 x

1번 단점 해결 대책 : TF-IDF "희귀성을 고려"

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer=CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer=TfidfTransformer()

tf= vectorizer.fit_transform(df['text'])
tfidf=transformer.fit_transform(tf)
print(tfidf.toarray())

[[0.         0.31544415 0.40619178 0.         0.40619178 0.
  0.         0.53409337 0.         0.         0.53409337]
 [0.43306685 0.33631504 0.43306685 0.         0.43306685 0.
  0.         0.         0.56943086 0.         0.        ]
 [0.34261996 0.26607496 0.         0.45050407 0.         0.45050407
  0.45050407 0.         0.         0.45050407 0.        ]]


In [44]:
print(list(tfidf.toarray()[0]))
print(list(tf.toarray()[0]))

print('index 비교')
for i, j in zip(list(tfidf.toarray()[0]), list(tf.toarray()[0])):
    print("tfid: ", i)
    print("tf: ",j)
    print("------------")

[0.0, 0.31544415103177975, 0.4061917781433947, 0.0, 0.4061917781433947, 0.0, 0.0, 0.5340933749435834, 0.0, 0.0, 0.5340933749435834]
[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1]
index 비교
tfid:  0.0
tf:  0
------------
tfid:  0.31544415103177975
tf:  1
------------
tfid:  0.4061917781433947
tf:  1
------------
tfid:  0.0
tf:  0
------------
tfid:  0.4061917781433947
tf:  1
------------
tfid:  0.0
tf:  0
------------
tfid:  0.0
tf:  0
------------
tfid:  0.5340933749435834
tf:  1
------------
tfid:  0.0
tf:  0
------------
tfid:  0.0
tf:  0
------------
tfid:  0.5340933749435834
tf:  1
------------


-> bag of words랑 index순서는 같음 <-> 값이 다름 (값이 클수록 희귀도가 높음)

2번 단점 해결 대책 : Word2vec "의미 유사도를 벡터로 생성"

In [45]:
from gensim.models import word2vec

sentences=[d.split() for d in df['text']]   # 각 문장을 단어로 쪼갠 list 3개가 sentences로 들어감
model = word2vec.Word2Vec(sentences, vector_size=10, min_count=1, window=2, seed=7)
# 교재에는 size로 나와있으나 최신버전에서 vector_size로 변경됨

In [46]:
model.wv['like']        # 벡터 형식으로 변환된 'like' 벡터 (10차원)

array([ 0.01650858,  0.01069946,  0.00188946,  0.09910005,  0.06153275,
        0.05853238,  0.04005488,  0.02443584, -0.03179482,  0.09779203],
      dtype=float32)

In [47]:
model.wv.most_similar('like')   # 'like'단어 벡터와 유사한 벡터 순서대로 출력

[('I', 0.42540043592453003),
 ('machine', 0.36355969309806824),
 ('not', 0.311229407787323),
 ('kaggle', -0.004140517208725214),
 ('much', -0.11530755460262299),
 ('do', -0.1529018133878708),
 ('love', -0.2554278075695038),
 ('really', -0.4161785840988159),
 ('learning', -0.44330504536628723),
 ('very', -0.44338396191596985)]

In [48]:
import numpy as np

wordvec = np.array([model.wv[word] for word in df['text'][0].split()])
# text 데이터 프레임의 첫문장을 단어 단위로 쪼갬 -> 각 단어마다 10차원 벡터로 변환 -> wordvec이라는 배열에 추가
wordvec

array([[ 0.08898099,  0.02501909,  0.03683598,  0.07944275,  0.01565849,
         0.05513714,  0.0667302 , -0.05495857, -0.08889369, -0.03996675],
       [ 0.01650858,  0.01069946,  0.00188946,  0.09910005,  0.06153275,
         0.05853238,  0.04005488,  0.02443584, -0.03179482,  0.09779203],
       [ 0.06329302, -0.03939352, -0.03167932, -0.04431488,  0.04389417,
        -0.04902608,  0.09809195, -0.01098474, -0.00437022,  0.00090965],
       [ 0.03720424, -0.02774719,  0.02864924,  0.01963681, -0.07835456,
        -0.08814968,  0.03203132, -0.02247364,  0.01966591, -0.03539274],
       [-0.09157717,  0.04835419, -0.00529734, -0.08170088, -0.05110302,
         0.00822875,  0.04535742,  0.00155444,  0.02258943,  0.07426786]],
      dtype=float32)

=> text 데이터 프레임의 첫문장의 단어들을 모두 10차원 벡터로 변환한 array wordvet생성

벡터화한 단어들을 머신러닝 알고리즘에 입력

1. 문장에 등장하는 단어 벡터의 평균값
2. 문장에 등장하는 단어 벡터 각 요소의 최댓값
3. 각 단어를 시계열 데이터로 다룸 "RNN, BERT"

< 평균값>

In [49]:
np.mean(wordvec, axis=0) #5개의 10차원 단어 벡터를 각 요소(차원)마다 평균값 계산

array([ 0.02288193,  0.00338641,  0.0060796 ,  0.01443277, -0.00167443,
       -0.0030555 ,  0.05645315, -0.01248533, -0.01656068,  0.01952201],
      dtype=float32)

<최댓값> "SWEM-max"

In [50]:
np.max(wordvec, axis=0)

array([0.08898099, 0.04835419, 0.03683598, 0.09910005, 0.06153275,
       0.05853238, 0.09809195, 0.02443584, 0.02258943, 0.09779203],
      dtype=float32)